In [1]:
import tensorflow as tf

# Create TensorFlow object called tensor
hello_constant = tf.constant('Hello World!')

with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

b'Hello World!'


In [4]:
x = tf.placeholder(tf.string)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Hello World'})
    print(output)
    output = sess.run(hello_constant)
    print(output)

Hello World
b'Hello World!'


In [10]:
y = tf.multiply(2, 5)  # 10
with tf.Session() as sess:
    output = sess.run(y)
    print(output)

10


In [16]:
y = tf.subtract(tf.cast(tf.constant(2.0), tf.int32),tf.constant(1)) 
with tf.Session() as sess:
    output = sess.run(y)
    print(output)

1


In [20]:
'''
x= 10
y  =2
z = x/y - 1'''

x = tf.constant(10)
y = tf.constant(2)
z = tf.subtract(tf.divide(x, y), tf.cast(tf.constant(1), tf.float64))

with tf.Session() as sess:
    output = sess.run(z)
    print(output)

4.0


In [23]:
x = tf.Variable(5)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

In [25]:
n_feature = 120
n_labels = 5

weights = tf.Variable(tf.truncated_normal((n_feature, n_labels)))
bias = tf.Variable(tf.zeros(n_labels))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

In [50]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [51]:
def get_weights(n_features, n_labels):
    return tf.Variable(tf.truncated_normal((n_features, n_labels)))

def get_bias(n_labels):
    return tf.Variable(tf.zeros(n_labels))

def linear(inputs, weight, bias):
    return tf.add(tf.matmul(inputs, weight), bias)

In [52]:
def mnist_features_labels(n_labels):
    mnist_features = []
    mnist_labels = []
    
    mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)
    
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])
    return mnist_features, mnist_labels

In [56]:
n_features = 784
n_labels = 3

features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

w = get_weights(n_features, n_labels)
b= get_bias(n_labels)

logits = linear(features, w, b)

train_features, train_labels = mnist_features_labels(n_labels)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    prediction = tf.nn.softmax(logits)
    
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
    
    loss = tf.reduce_mean(cross_entropy)
    
    learning_rate = 0.08
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    _,l = sess.run(
        [optimizer, loss],
        feed_dict={features:train_features, labels:train_labels})
    print(cross_entropy.value_index)

print("Loss", l)




Extracting /datasets/ud730/mnist\train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-labels-idx1-ubyte.gz
0
Loss 8.7947


In [71]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    num_batches = len(labels) // batch_size
    last_batch = len(labels) % batch_size
    batch = []*(num_batches + 1)
    i = 0
    while(i < num_batches):
        chunk = i * batch_size
        #print ("i {} chunk size {}".format(i, chunk))
        batch.append([features[chunk:chunk+batch_size], labels[chunk:chunk+batch_size]])
        i += 1
    chunk = i * last_batch
    batch.append([features[chunk:chunk+last_batch], labels[chunk:chunk+last_batch]])
    return batch

In [68]:
from pprint import pprint

# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]
# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

# PPrint prints data structures like 2d arrays, so they are easier to read
pprint(batches(3, example_features, example_labels))


i 0 chunk size 0
[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F21', 'F22', 'F23', 'F24']], [['L21', 'L22']]]]


In [ ]:
def epochs_stat(epoch_i, sess, last_features, last_labels):
    sess.run(cost, feed_dict={features:last_features. labels:last_labels})
    valid_acc = sess.run(accuracy, )

In [73]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)


# Import MNIST data
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)


# The features are already scaled and the data is shuffled
train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

weights = tf.Variable(tf.truncated_normal([n_input, n_classes]))
bias = tf.Variable(tf.truncated_normal([n_classes]))

logits = tf.add(tf.matmul(features, weights), bias)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = labels, logits=logits))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

batch_size = 128

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    batch = batches(batch_size, train_features, train_labels)
    
    for batch_features, batch_labels in batch:
        sess.run(optimizer, feed_dict={features:batch_features, labels:batch_labels})

    test_acc = sess.run(accuracy, feed_dict={features:test_features, labels:test_labels})
print("Test Accuracy {}".format(test_acc))



Extracting /datasets/ud730/mnist\train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-labels-idx1-ubyte.gz
Test Accuracy 0.06499999761581421
